In [ ]:
!pip install rdflib
!pip install PyDrive

In [2]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, OWL, XSD, FOAF, DCTERMS, VOID
from rdflib.serializer import Serializer

In [3]:
import collections
import pandas as pd

In [4]:
df = pd.DataFrame(pd.read_csv("apart.csv"))
df.drop(['Unnamed: 0'], axis='columns', inplace=True)
dt = df[570:580]

In [54]:
g = Graph()
g.parse("ofkop.owl",  format='turtle')
g


<Graph identifier=N62296876afc54932a9bb67c677be0acb (<class 'rdflib.graph.Graph'>)>

In [49]:
prefix = dict(g.namespaces())['']
ns = Namespace(prefix)
ns

Namespace('http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#')

Заполняем граф значениями классов и триплетов 

In [55]:
for index, row in df.iterrows():    
    
    apart = URIRef(ns + str(row[0])) # id кв
    g.add((apart, RDF.type, OWL.NamedIndividual))
    g.add((apart, RDF.type, ns.Apartment))
    
    discription = Literal(row['Описание'], datatype=XSD.Literal) # 
    link = Literal(row['Ссылка на объявление'], datatype=XSD.Literal) 
    g.add((apart, RDFS.comment, discription))
    g.add((apart, RDFS.seeAlso, link))
    
    contacts = URIRef(ns + "Contacts")
    phone = URIRef(ns + Literal(row['Телефоны'], datatype=XSD.Literal))
    g.add((contacts, RDF.type, ns.Contacts))
    g.add((phone, RDF.type, ns.Phone))
    g.add((apart, ns.contactBy, phone))
    g.add((contacts, ns.contactBy, phone))

    metro = URIRef(ns + (str(row['Метро']).replace(" ", "_")))
    timeToMetro = Literal(row['Время до метро, мин'], datatype=XSD.int)
    wayToMetro = Literal(str(row['Способ до метро']), datatype=XSD.string)
    g.add((metro, RDF.type, OWL.NamedIndividual))
    g.add((metro, RDF.type, ns.Metro))
    g.add((apart, ns.nextToMetro, metro)) # метро
    g.add((apart, ns.TimeToMetro, timeToMetro)) # время до метро
    g.add((apart, ns.WayToMetro, wayToMetro)) # путь до метро

    address = URIRef(ns + (str(row['Адрес'].replace(",", "").replace(" ", "_"))))
    city = URIRef(ns + Literal(row['Город'], datatype=XSD.string))
    street = URIRef(ns + Literal(row['Улица'], datatype=XSD.string).replace(" ", "_"))
    numHouse = URIRef(ns + Literal(row['Номер дома'], datatype=XSD.string))
    g.add((address, RDF.type, OWL.NamedIndividual))
    g.add((address, RDF.type, ns.Address))
    g.add((city, RDF.type, ns.City))
    g.add((street, RDF.type, ns.Street))
    g.add((numHouse, RDF.type, ns.HouseNumber))
    g.add((address, ns.consistsCity, city))
    g.add((address, ns.consistsStreet, street))
    g.add((address, ns.consistsNum, numHouse))
    g.add((apart, ns.hasAddress, address)) 
   
    rub = "http://www.ontologyrepository.com/CommonCoreOntologies/RussianRouble"
    price = URIRef(ns + str("Price_" + str(row[0])))
    rental = Literal(row['Цена аренды, руб/мес'], datatype=XSD.float)
    bills = Literal(row['Ком платежи'], datatype=XSD.float)
    deposit =  Literal(row['Залог'], datatype=XSD.float)
    currency = URIRef(ns + Literal(rub))
    g.add((price, RDF.type, OWL.NamedIndividual))
    g.add((price, RDF.type, ns.Price))
    g.add((currency, RDF.type, ns.Currency))
    g.add((price, ns.Rental, rental))
    g.add((price, ns.Bills, bills))
    g.add((price, ns.Deposit, deposit))
    g.add((price, ns.hasCurrency, currency))
    g.add((apart, ns.hasPrice, price))

    appliances = URIRef(ns + str("Appliances_" + str(row[0])))
    washer = URIRef(ns + (str(row['Стиральная машина'])))
    dishwasher = URIRef(ns + (str(row['Посудомоечная машина'])))
    tv = URIRef(ns + (str(row['Телевизор'])))
    fridge = URIRef(ns + (str(row['Холодильник'])))
    router = URIRef(ns + (str(row['Интернет'])))
    g.add((appliances, RDF.type, OWL.NamedIndividual))
    g.add((appliances, RDF.type, ns.Appliances))
    g.add((washer, RDF.type, ns.Washer))
    g.add((dishwasher, RDF.type, ns.Dishwasher))
    g.add((tv, RDF.type, ns.TV))
    g.add((fridge, RDF.type, ns.Fridge))
    g.add((router, RDF.type, ns.Router))
    g.add((apart, ns.hasAppliances, appliances)) 
    g.add((appliances, ns.hasWasher, washer)) 
    g.add((appliances, ns.hasDishwasher, dishwasher)) 
    g.add((appliances, ns.hasTV, tv))
    g.add((appliances, ns.hasFridge, fridge)) 
    g.add((appliances, ns.hasRouter, router))

    fur = URIRef(ns + "Furniture")
    furBath = URIRef(ns + (str(row['Ванна,'])))
    furKit = URIRef(ns + (str(row['Мебель на кухне'])))
    furRoom = URIRef(ns + (str(row['Мебель в комнатах'])))
    furSh = URIRef(ns + (str(row['Душевая кабина'])))
    g.add((fur, RDF.type, ns.Furniture))
    g.add((furBath, RDF.type, ns.Bathroom))
    g.add((furKit, RDF.type, ns.Kitchen))
    g.add((furRoom, RDF.type, ns.Rooms))
    g.add((furSh, RDF.type, ns.ShowerRoom))
    g.add((apart, ns.hasFurnitureBathroom, furBath))
    g.add((apart, ns.hasFurnitureKitchen, furKit))
    g.add((apart, ns.hasFurnitureRooms, furRoom))
    g.add((apart, ns.hasFurnitureShower, furSh))

    repair = URIRef(ns + (str(row['Ремонт']).replace(" ", "_")))
    windows = URIRef(ns + (str(row['Окна']).replace(" ", "_")))
    child = Literal(row['Можно с детьми'], datatype=XSD.bool)  
    aminal = Literal(row['Можно с животными'], datatype=XSD.bool)  
    numBalcony = Literal(row['Балкон'], datatype=XSD.int)
    numLoggia = Literal(row['Лоджия'], datatype=XSD.int)
    numRoom = Literal(row['Количество комнат'], datatype=XSD.int)
    floor = Literal(row['Этаж'], datatype=XSD.int)
    square = Literal(float(row['Площадь, м2']), datatype=XSD.float)
    g.add((repair, RDF.type, ns.Repair))
    g.add((windows, RDF.type, ns.Windows))
    g.add((apart, ns.PossibleChild, child)) 
    g.add((apart, ns.PossibleAnimal, aminal))
    g.add((apart, ns.NumBalcony, numBalcony))
    g.add((apart, ns.NumLoggia, numLoggia))
    g.add((apart, ns.NumRoom, numRoom))
    g.add((apart, ns.Floor, floor))
    g.add((apart, ns.Square, square))

    nameGK = URIRef(ns + (str(row['Название ЖК']).replace(" ", "_")))
    park = URIRef(ns + (str(row['Парковка'])))
    numFreLift = Literal(row['Лифт груз'], datatype=XSD.int)
    numPassLift = Literal(row['Лифт пасс'], datatype=XSD.int)
    g.add((nameGK, RDF.type, ns.NameOfComplex))
    g.add((park, RDF.type, ns.Parking))
    g.add((apart, ns.NumFreLift, numFreLift))
    g.add((apart, ns.NumPassLift, numPassLift))
    g.add((apart, ns.hasPark, park))
    g.add((apart, ns.hasComplex, nameGK))
    
   # distance = Literal(float(row['Дист до а1']), datatype=XSD.float)   
   # address1 = Literal(str("Кронверкский проспект, 49, Санкт-Петербург"), datatype=XSD.Literal)
   # g.add((distance, RDFS.comment, address1)) 
   # g.add((apart, ns.Distance, distance))



In [57]:
g.serialize(destination='hehe.owl', format='turtle')

In [ ]:
print(g.serialize(format="turtle").decode("utf-8"))

SPARQL

Где в Санкт-Петербурге больше шансав снять квартиру квартиранту с кошкой?

In [ ]:
q = g.query(
    """PREFIX : <http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#>
    SELECT ?metro (count(?metro) as ?metrocount)
    WHERE {
        ?apart a :Apartment;
            :nextToMetro ?metro;
            :PossibleAnimal ?animal;
          FILTER ( ?animal = "true"^^xsd:bool )    
    } GROUP BY ?metro
      ORDER BY DESC(?metrocount)
      LIMIT 10
    """)
for row in q:
   print(row[1] + " предложения на " +row[0].split('#')[1])

Какое изменение цены однокомнатной квартиры в зависимости от района? 

In [ ]:
q = g.query(
    """PREFIX : <http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#>
    SELECT  ?metro ?rental (AVG(?rental) AS ?average_rental) ?currency
    WHERE {
      ?apart a :Apartment;
      :nextToMetro ?metro;
      :NumRoom ?numRoom;
      :hasPrice ?price;
          {
            SELECT ?rental ?currency
            WHERE {
              ?price a :Price;
              :Rental ?rental;
              :hasCurrency ?currency;
                  }
          }              
    FILTER ( ?numRoom = "1"^^xsd:int )   
    } GROUP BY ?metro
      ORDER BY DESC(?average_rental)
      LIMIT 15
    """
    )
for row in q:
  #print(row)
  print("Средняя цена 1 комнт. кв. " + row[0].split('#')[1] + "  " + row[2])

Какие станции самые популярные по добавлению объявлений?

In [ ]:
q = g.query(
    """PREFIX : <http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#>
    SELECT ?metro (count(?metro) as ?metrocount)
    WHERE {
        ?apart a :Apartment;
            :nextToMetro ?metro;  
    } GROUP BY ?metro
      ORDER BY DESC(?metrocount)
      LIMIT 15
    """)
for row in q:
   print(row[1] + " предложения на " +row[0].split('#')[1])

Какие предложения наиболее подходяшие для молодой семьи с двумя детьми?

In [ ]:
q = g.query(
    """PREFIX : <http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#>
    SELECT ?apart ?sum_price ?rental ?bills
    WHERE {
        ?apart rdf:type :Apartment;
        :PossibleChild ?child;
        :NumRoom ?numRoom;
        :Square ?square;
        :WayToMetro ?wayToMetro;
        :TimeToMetro ?timeToMetro;
        :hasPrice ?price;
        {
            SELECT ?sum_price ?rental ?bills
            WHERE {
              ?price a :Price;
              :Rental ?rental;
              :Bills ?bills;
              BIND( ?rental + ?bills AS ?sum_price) 
              FILTER (?rental < 100000)
                  }
          }
         FILTER (?sum_price < 100000)
         FILTER (?numRoom >= "1"^^xsd:int 
         && ?square > "100"^^xsd:float
         && ?child = "true"^^xsd:bool
         && ?timeToMetro <= "7"^^xsd:int
         && ?wayToMetro = "пешком"^^xsd:string)              
    }
        LIMIT 10
    """)
for row in q:
    print("https://spb.cian.ru/rent/flat/" + row[0].split('#')[1] + " " +  row[1] + " : аренда " +  row[2] + ", ку " +  row[3])
    print(str(g.comment(row[0]))+'\n')

Какие квартиры в диапазоне N расположенны максимально близко (расстояние в км) к локации M? 



In [ ]:
# Кронверкский проспект, 49, Санкт-Петербург
q = g.query(
    """PREFIX : <http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15#>
    SELECT ?apart ?distance ?metro ?rental   
    WHERE {
        ?apart a :Apartment;
        :Distance ?distance;
        :nextToMetro ?metro; 
        :hasPrice ?price;
        {
          SELECT ?rental
          WHERE {
            ?price a :Price;
            :Rental ?rental;
            FILTER (?rental < 100000)
                }
          }
    FILTER (?distance < 2 )
    }
    ORDER BY ?distance
    LIMIT 15
    """
    )
for row in q:
    #print(row)
    print("https://spb.cian.ru/rent/flat/" + row[0].split('#')[1] + "\n" + row[1] + " " +  row[2].split('#')[1] + " " +  row[3])
    #print(str(g.comment(row[0]))+'\n')

VoID

In [51]:
g = Graph()
g.parse("ofkop.owl",  format='turtle')
g

<Graph identifier=N07659576c87a489f9ba35899faf56411 (<class 'rdflib.graph.Graph'>)>

In [56]:
g.add((prefix, DCTERMS.title, Literal('Residential property rental')))
g.add((prefix, DCTERMS.publisher, Literal('Ekaterina', datatype=FOAF.Person)))
g.add((prefix, DCTERMS.publisher, Literal('trfnthbyf997@gmail.com', datatype=FOAF.mbox)))
g.add((prefix, DCTERMS.issued, Literal('2021-04-23', datatype=XSD.data)))
g.add((prefix, VOID.triples, Literal(int(47820))))
g.add((prefix, VOID.classes, Literal(int(31))))
g.add((prefix, VOID.properties, Literal(int(45))))
g.add((prefix, VOID.distinctObjects, Literal(int(6842))))
g.add((prefix, VOID.distinctSubjects, Literal(int(5227))))

In [ ]:
print(g.serialize(format="turtle").decode("utf-8"))

In [43]:
distinctForPartitions = True
res = Graph()

In [44]:
def generateVoID(g):
    typeMap = collections.defaultdict(set)
    classes = collections.defaultdict(set)
    for e, c in g.subject_objects(RDF.type):
        classes[c].add(e)
        typeMap[e].add(c)

    triples = 0
    subjects = set()
    objects = set()
    properties = set()
    classCount = collections.defaultdict(int)
    propCount = collections.defaultdict(int)

    classProps = collections.defaultdict(set)
    classObjects = collections.defaultdict(set)
    propSubjects = collections.defaultdict(set)
    propObjects = collections.defaultdict(set)

    for s, p, o in g:

        triples += 1
        subjects.add(s)
        properties.add(p)
        objects.add(o)

        # class partitions
        if s in typeMap:
            for c in typeMap[s]:
                classCount[c] += 1
                if distinctForPartitions:
                    classObjects[c].add(o)
                    classProps[c].add(p)

        # property partitions
        propCount[p] += 1
        if distinctForPartitions:
            propObjects[p].add(o)
            propSubjects[p].add(s)

    dataset = URIRef("http://www.semanticweb.org/asususer/ontologies/2021/3/untitled-ontology-15")


    res.add((dataset, RDF.type, VOID.Dataset))

    # basic stats
    res.add((dataset, VOID.triples, Literal(triples)))
    res.add((dataset, VOID.classes, Literal(len(classes))))

    res.add((dataset, VOID.distinctObjects, Literal(len(objects))))
    res.add((dataset, VOID.distinctSubjects, Literal(len(subjects))))
    res.add((dataset, VOID.properties, Literal(len(properties))))

    for i, c in enumerate(classes):
        part = URIRef(dataset + "_class%d" % i)
        res.add((dataset, VOID.classPartition, part))
        res.add((part, RDF.type, VOID.Dataset))

        res.add((part, VOID.triples, Literal(classCount[c])))
        res.add((part, VOID.classes, Literal(1)))

        res.add((part, VOID["class"], c))

        res.add((part, VOID.entities, Literal(len(classes[c]))))
        res.add((part, VOID.distinctSubjects, Literal(len(classes[c]))))

        if distinctForPartitions:
            res.add(
                (part, VOID.properties, Literal(len(classProps[c]))))
            res.add((part, VOID.distinctObjects,
                    Literal(len(classObjects[c]))))

    for i, p in enumerate(properties):
        part = URIRef(dataset + "_property%d" % i)
        res.add((dataset, VOID.propertyPartition, part))
        res.add((part, RDF.type, VOID.Dataset))

        res.add((part, VOID.triples, Literal(propCount[p])))
        res.add((part, VOID.properties, Literal(1)))

        res.add((part, VOID.property, p))

        if distinctForPartitions:

            entities = 0
            propClasses = set()
            for s in propSubjects[p]:
                if s in typeMap:
                    entities += 1
                for c in typeMap[s]:
                    propClasses.add(c)

            res.add((part, VOID.entities, Literal(entities)))
            res.add((part, VOID.classes, Literal(len(propClasses))))

            res.add((part, VOID.distinctSubjects,
                    Literal(len(propSubjects[p]))))
            res.add((part, VOID.distinctObjects,
                    Literal(len(propObjects[p]))))
    
    return res

In [ ]:
generateVoID(g)
print(res.serialize(format="turtle").decode("utf-8"))